In [34]:
import datetime
import pandas as pd
import numpy as np
import os
import optuna
from os.path import curdir
import math
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [35]:
data_path = ('train.csv')
df = pd.read_csv(data_path)

In [36]:
# Modify the header row to remove trailing spaces
df.columns = [header.strip() for header in df.columns]

In [37]:
df['EJ'] = df['EJ'].replace('A', '1')
df['EJ'] = df['EJ'].replace('B', '2')

In [38]:
df['EJ'] = df['EJ'].astype('float64')

In [39]:
# Calculate the mean for "BQ" column and fill missing values with the mean
mean_bq = df["BQ"].mean()
df["BQ"].fillna(mean_bq, inplace=True)


mean_cb = df["CB"].mean()
df["CB"].fillna(mean_cb, inplace=True)


mean_cc = df["CC"].mean()
df["CC"].fillna(mean_cc, inplace=True)

mean_DU = df["DU"].mean()
df['DU'].fillna(mean_DU, inplace=True)

mean_EL = df["EL"].mean()
df["EL"].fillna(mean_EL, inplace=True)

mean_FC = df["FC"].mean()
df["FC"].fillna(mean_FC, inplace=True)

mean_FL = df["FL"].mean()
df["FL"].fillna(mean_FL, inplace=True)

mean_FS = df["FS"].mean()
df["FS"].fillna(mean_FS, inplace=True)

mean_GL = df["GL"].mean()
df["GL"].fillna(mean_GL, inplace=True)

In [40]:
df

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,fd3dafe738fd,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,...,0.173229,1.26092,0.067730,8.967128,217.148554,8095.932828,24.640462,69.191944,21.978000,0
613,fd895603f071,0.435846,5462.03438,85.200147,46.551007,15.973224,5.979825,0.025882,12.622906,3.777550,...,10.223150,1.24236,0.426699,35.896418,496.994214,3085.308063,29.648928,124.808872,0.145340,0
614,fd8ef6377f76,0.427300,2459.10720,130.138587,55.355778,10.005552,8.070549,0.025578,15.408390,1.229900,...,0.173229,0.49706,0.067730,19.962092,128.896894,6474.652866,26.166072,119.559420,21.978000,0
615,fe1942975e40,0.363205,1263.53524,85.200147,23.685856,8.138688,7.981959,0.025578,7.524588,1.229900,...,9.256996,0.78764,0.670527,24.594488,72.611063,1965.343176,25.116750,37.155112,0.184622,0


In [41]:
df=df.drop(['Id'], axis=1)

In [42]:
#Normalize the Data

X = df.copy()
scaler = StandardScaler()
features = ['AB','AF','AH','AM','AR','AX', 'AY','AZ','BC','BD','BN','BP','BQ','BR','BZ','CB','CC','CD','CF','CH','CL','CR','CS','CU','CW','DA','DE','DF','DH','DI','DL',
 'DN','DU','DV','DY','EB','EE','EG','EH','EJ','EL','EP','EU','FC','FD','FE','FI','FL','FR','FS','GB','GE','GF','GH','GI','GL',]
X[features] = scaler.fit_transform(X[features])


In [43]:
X.shape
X.info()
X.describe().transpose()
X.head(10)
X.tail()

(617, 57)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 57 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AB      617 non-null    float64
 1   AF      617 non-null    float64
 2   AH      617 non-null    float64
 3   AM      617 non-null    float64
 4   AR      617 non-null    float64
 5   AX      617 non-null    float64
 6   AY      617 non-null    float64
 7   AZ      617 non-null    float64
 8   BC      617 non-null    float64
 9   BD      617 non-null    float64
 10  BN      617 non-null    float64
 11  BP      617 non-null    float64
 12  BQ      617 non-null    float64
 13  BR      617 non-null    float64
 14  BZ      617 non-null    float64
 15  CB      617 non-null    float64
 16  CC      617 non-null    float64
 17  CD      617 non-null    float64
 18  CF      617 non-null    float64
 19  CH      617 non-null    float64
 20  CL      617 non-null    float64
 21  CR      617 non-null    float64
 22  CS

,count,mean,std,min,25%,50%,75%,max
AB,617.0,-3.157365e-17,1.000811,-0.846058,-0.480851,-2.617267e-01,0.176522,12.146180
AF,617.0,-2.218647e-16,1.000811,-1.439844,-0.567627,-1.660654e-01,0.374000,10.957858
AH,617.0,-1.504289e-16,1.000811,-0.261669,-0.261669,-2.616690e-01,-0.038243,14.025086
AM,617.0,1.871365e-17,1.000811,-0.513710,-0.383201,-2.646045e-01,0.002459,8.490531
AR,617.0,-4.759384e-17,1.000811,-0.189295,-0.189295,-1.892948e-01,-0.189295,16.061825
AX,617.0,-3.753525e-16,1.000811,-1.900558,-0.555878,-2.014664e-01,0.347525,12.835330
AY,617.0,-1.862368e-16,1.000811,-0.083417,-0.083417,-8.341747e-02,-0.056366,24.624368
AZ,617.0,-8.097251e-17,1.000811,-1.649292,-0.560571,-2.418314e-02,0.552796,6.534241
BC,617.0,-2.868374e-17,1.000811,-0.104787,-0.104787,-1.047870e-01,-0.045639,22.355222
BD,617.0,-5.542118e-17,1.000811,-1.211299,-0.395738,-1.167414e-01,0.227070,15.803958


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,-0.572153,-0.170975,-0.261669,-0.237889,-0.189295,-1.900558,-0.083417,-0.173502,-0.038354,-0.405383,...,0.162487,-0.035806,-0.250869,-0.940094,-0.410260,-0.655511,-0.948991,0.531241,-0.815752,1
1,-0.709105,-1.097801,-0.261669,-0.028701,-0.189295,-0.750457,-0.083417,0.678919,-0.104787,0.048541,...,-0.458281,-0.060566,0.113218,-1.145070,-0.410260,0.687893,-0.238862,-0.509218,1.304239,0
2,-0.015212,-0.377169,-0.261669,-0.094845,-0.189295,0.465662,-0.083417,0.519453,-0.104787,-0.071089,...,0.198331,-0.051023,0.596934,1.637944,-0.299210,-0.051850,-0.351743,-0.424754,-0.808323,0
3,-0.480851,0.138196,0.012347,0.547477,-0.189295,-0.729610,-0.083417,0.112088,-0.104787,-0.391109,...,0.060027,-0.060566,-0.105234,-0.219883,-0.342195,-0.650833,0.858232,1.101332,-0.812311,0
4,-0.206946,0.100517,-0.261669,-0.356885,-0.189295,-0.628845,-0.013229,-1.649292,1.445139,0.125327,...,0.236971,0.896815,-0.230064,-0.432313,0.099920,-0.318309,1.409422,-0.395228,-0.818054,1
5,-0.572153,-0.385562,-0.261669,-0.436720,-0.189295,-0.601048,-0.083417,0.455666,-0.104787,-0.037381,...,-0.458281,-0.060566,0.570928,0.119259,-0.410260,0.491176,-0.300777,0.881515,1.304239,0
6,-0.275422,-0.769368,-0.261669,-0.439076,0.493251,-1.424535,-0.083417,-0.924444,-0.104787,0.119274,...,-0.089279,-0.053302,0.035199,-0.285103,0.421603,-0.586801,0.271554,1.366078,-0.821111,0
7,-0.444330,-1.103156,-0.261669,-0.255403,-0.189295,-0.218839,-0.083417,-0.266282,-0.104787,-0.102558,...,0.100953,-0.060566,-0.110436,-0.228269,-0.152855,0.346076,0.377262,-0.528736,-0.818417,0
8,-0.279987,-0.114675,-0.261669,-0.144675,-0.189295,-0.597574,-0.083417,-0.534476,-0.067982,0.403786,...,-0.058454,-0.046893,-0.271674,-0.348459,0.107616,-0.521789,-0.174305,0.112900,-0.820304,0
9,-0.325638,0.733826,-0.261669,-0.373177,-0.189295,-0.373460,-0.083417,0.027281,-0.104787,-0.132066,...,-0.458281,-0.039137,-0.079228,0.379671,-0.227573,-0.471445,-0.963337,-0.476958,1.304239,0


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
612,-0.699975,-0.161828,0.040232,-0.422762,0.275215,-0.802577,0.040875,-0.464891,-0.080610,-0.395082,...,-0.458281,-0.045332,-0.271674,-1.177680,0.593022,-0.340465,-0.694915,0.513497,1.304239,0
613,-0.088253,0.852755,-0.261669,0.108831,0.556117,0.170319,-0.082686,0.473063,-0.065661,0.100596,...,0.417330,-0.045702,0.003992,1.519617,2.535523,-0.599582,-0.186764,2.048314,-0.813328,0
614,-0.106514,-0.453742,0.090140,0.235206,-0.011673,0.990330,-0.083417,1.113828,-0.104787,0.178374,...,-0.458281,-0.060566,-0.271674,-0.076400,-0.019561,-0.424307,-0.540129,1.903449,1.304239,0
615,-0.243466,-0.973904,-0.261669,-0.219353,-0.189295,0.955584,-0.083417,-0.699741,-0.104787,-0.275772,...,0.333153,-0.054771,0.191237,0.387591,-0.410260,-0.657500,-0.646592,-0.370599,-0.809518,0
616,0.012178,-0.360885,3.350987,1.048310,-0.189295,-0.920714,0.135921,-1.649292,-0.001602,-0.838845,...,-0.458281,-0.047645,-0.209259,-0.706235,-0.410260,-0.404872,1.446419,1.773276,1.304239,0


In [44]:
from sklearn.model_selection import train_test_split
y=df['Class']
X=df.drop(columns=['Class'])
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)

In [45]:
#Random Forests

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from scipy.stats import randint as sp_randint

clf_rf = RandomForestClassifier()


params = {'n_estimators': sp_randint(10, 200),  # Randomly select a number between 10 and 200
         'max_depth': sp_randint(1, 20),
         'min_samples_split': sp_randint(2, 20),
         'min_samples_leaf': sp_randint(1, 20),
         'max_features': ['auto', 'sqrt', 'log2'],
         'bootstrap': [True, False]}

n_iter_search = 20  # Number of random combinations to try
random_search = RandomizedSearchCV(clf_rf, param_distributions=params,
                                   n_iter=n_iter_search, cv=10, random_state=42)
random_search.fit(X_train, y_train)




RandomizedSearchCV(cv=10, estimator=RandomForestClassifier(), n_iter=20,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fc95cf7c7c0>,
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fc956044df0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fc95ceaadc0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fc95ceaa160>},
                   random_state=42)

In [47]:
# After fitting the Random Search object, access the best hyperparameters and the best estimator as follows:
print("Best hyperparameters:", random_search.best_params_)
print("Best estimator:", random_search.best_estimator_)

Best hyperparameters: {'bootstrap': True, 'max_depth': 7, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 13, 'n_estimators': 145}
Best estimator: RandomForestClassifier(max_depth=7, min_samples_split=13, n_estimators=145)


In [51]:
test_df = pd.read_csv('test.csv')
#pd.read_csv('test.csv',index_col="Id")
test_df

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
test_df_copy = test_df.set_index("Id").copy()

In [54]:
test_df_copy 


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
Id,,,,,,,,,,,,,,,,,,,,,
00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
test_df_copy['EJ'] = test_df_copy['EJ'].replace('A', '1')
test_df_copy['EJ'] = test_df_copy['EJ'].replace('B', '2')

In [56]:
test_df_copy['EJ'] = test_df_copy['EJ'].astype('float64')

In [57]:
# Calculate the mean for "BQ" column and fill missing values with the mean
mean_bq = test_df_copy["BQ"].mean()
test_df_copy["BQ"].fillna(mean_bq, inplace=True)


mean_cb = test_df_copy["CB"].mean()
test_df_copy["CB"].fillna(mean_cb, inplace=True)


mean_cc = test_df_copy["CC"].mean()
test_df_copy["CC"].fillna(mean_cc, inplace=True)

mean_DU = test_df_copy["DU"].mean()
test_df_copy['DU'].fillna(mean_DU, inplace=True)

mean_EL = test_df_copy["EL"].mean()
test_df_copy["EL"].fillna(mean_EL, inplace=True)

mean_FC = test_df_copy["FC"].mean()
test_df_copy["FC"].fillna(mean_FC, inplace=True)

mean_FL = test_df_copy["FL"].mean()
test_df_copy["FL"].fillna(mean_FL, inplace=True)

mean_FS = test_df_copy["FS"].mean()
test_df_copy["FS"].fillna(mean_FS, inplace=True)

mean_GL =test_df_copy["GL"].mean()
test_df_copy["GL"].fillna(mean_GL, inplace=True)

In [59]:
#Normalize the Data

X = test_df_copy.copy()
scaler = StandardScaler()
features = ['AB','AF','AH','AM','AR','AX', 'AY','AZ','BC','BD ','BN','BP','BQ','BR','BZ','CB','CC','CD ','CF','CH','CL','CR','CS','CU','CW ','DA','DE','DF','DH','DI','DL',
 'DN','DU','DV','DY','EB','EE','EG','EH','EJ','EL','EP','EU','FC','FD ','FE','FI','FL','FR','FS','GB','GE','GF','GH','GI','GL',]
X[features] = scaler.fit_transform(X[features])

In [60]:
test_df_copy.columns = test_df_copy.columns.str.strip()

In [61]:
# Extract IDs from the test dataset
test_ids = test_df['Id']

# Make predictions
predictions = random_search.best_estimator_.predict_proba(test_df_copy)


# Create a new DataFrame with prediction results
prediction_df = pd.DataFrame({'Id': test_ids, 'class_0': predictions[:, 0], 'class_1': predictions[:, 1]})

# Print the prediction DataFrame
print(prediction_df)

             Id   class_0   class_1
0  00eed32682bb  0.502429  0.497571
1  010ebe33f668  0.502429  0.497571
2  02fa521e1838  0.502429  0.497571
3  040e15f562a2  0.502429  0.497571
4  046e85c7cc7f  0.502429  0.497571
